In [18]:
pip install pywhatkit selenium --q


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Users/sudarshan/.local/pipx/venvs/notebook/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Dependencies
import pywhatkit as pwk
from time import sleep
import sys
import webbrowser as wb
import time
import datetime

# PyAutoGUI implementation (commented out)
# from pyautogui import *

# Constants and variables
contacts = ['Varun']
message = 'Hello'
phone_no_list = ['+917875097430']
msg = """test!!!!"""

# Get current time + 1 minute
now = datetime.datetime.now()
_hr = now.hour
_min = now.minute + 1
if _min >= 60:
    _min = 0
    _hr += 1
    if _hr >= 24:
        _hr = 0

# PyAutoGUI functions (requires PyAutoGUI to be imported)
def click_search_name(name):
    """Find and click the search bar, then type a name"""
    x1, y1 = [211, 250]
    moveTo(x1, y1)
    click()
    typewrite(name, interval=0.2)
    sleep(2)
    press('enter')

def click_send_message(msg):
    """Find and click the message box, then type and send a message"""
    x3, y3 = [950, 750]
    moveTo(x3, y3)
    click()
    sleep(2)
    now = datetime.datetime.now()
    hr = now.hour
    minutes = now.minute + 1
    if minutes >= 60:
        minutes = 0
        hr += 1
        if hr >= 24:
            hr = 0
    press('enter')

# PyWhatKit methods
def send_scheduled_message():
    """Send a WhatsApp message at a scheduled time"""
    # Schedule message to be sent 1 minute from now
    hr, minutes = datetime.datetime.now().hour, datetime.datetime.now().minute + 1
    pwk.sendwhatmsg(
        '+917875097430', 
        msg, 
        time_hour=hr, 
        time_min=minutes, 
        wait_time=2, 
        tab_close=True
    )

def send_instant_messages():
    """Send instant messages to a list of phone numbers"""
    for ph_no in phone_no_list:
        pwk.sendwhatmsg_instantly(
            phone_no=ph_no,
            message='Test!!!'
        )

# Main execution
def main():
    # Options for sending WhatsApp messages:
    
    # Option 1: Send a scheduled message (will open browser automatically)
    # send_scheduled_message()
    
    # Option 2: Send instant messages to a list of phone numbers
    send_instant_messages()
    
    # Option 3: Send to a group (uncomment to use)
    # group_id = "FamilyGroup"
    # group_message = "Hello everyone!"
    # pwk.sendwhatmsg_to_group(group_id, group_message, _hr, _min + 1)
    
    # Option 4: Send an image with caption
    # image_path = "path/to/image.jpg"
    # caption = "Check out this photo!"
    # pwk.sendwhats_image("+917875097430", image_path, caption)
    
    print("Message scheduled/sent successfully!")

if __name__ == "__main__":
    main()

Message scheduled/sent successfully!


In [10]:
# Dependencies
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import urllib.parse
import os
import json
import datetime

# Constants
SESSION_INFO_FILE = "whatsapp_session_info.json"
SESSION_EXPIRY_DAYS = 14  # WhatsApp sessions typically last around 14 days

def get_session_info():
    """Get WhatsApp session information from the JSON file"""
    if os.path.exists(SESSION_INFO_FILE):
        try:
            with open(SESSION_INFO_FILE, 'r') as file:
                return json.load(file)
        except:
            return {"last_auth_date": None}
    return {"last_auth_date": None}

def save_session_info(session_info):
    """Save WhatsApp session information to the JSON file"""
    with open(SESSION_INFO_FILE, 'w') as file:
        json.dump(session_info, file)

def is_session_valid():
    """Check if the WhatsApp session is still valid"""
    session_info = get_session_info()
    last_auth_date = session_info.get("last_auth_date")
    
    if not last_auth_date:
        return False
    
    # Convert string date to datetime object
    last_auth = datetime.datetime.fromisoformat(last_auth_date)
    
    # Check if session is expired (older than SESSION_EXPIRY_DAYS)
    current_date = datetime.datetime.now()
    delta = current_date - last_auth
    
    return delta.days < SESSION_EXPIRY_DAYS

def initialize_whatsapp_session(force=False):
    """
    Initialize WhatsApp Web session with visible browser to scan QR code
    
    Args:
        force: If True, forces reinitialization even if session seems valid
    
    Returns:
        bool: True if initialization successful, False otherwise
    """
    # Check if session is still valid and not forced reinitialization
    if not force and is_session_valid():
        print("Session is still valid. No need to reinitialize.")
        return True
    
    print("Initializing WhatsApp Web session...")
    
    # Setup Chrome options - NOT headless for initial setup
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("user-data-dir=./chrome_profile")
    
    # Initialize the driver
    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        # Open WhatsApp Web
        driver.get("https://web.whatsapp.com/")
        
        # Wait for user to scan QR code and for WhatsApp to load
        print("\n" + "="*50)
        print("Please scan the QR code in the browser window that opened.")
        print("After scanning, wait until WhatsApp Web fully loads.")
        print("="*50 + "\n")
        
        # Wait for user avatar to appear indicating successful login
        try:
            # Increased timeout to 120 seconds
            WebDriverWait(driver, 120).until(
                EC.presence_of_element_located((By.XPATH, "//div[@data-testid='menu-bar-user-avatar']"))
            )
            print("QR code successfully scanned and WhatsApp Web loaded!")
            
            # Update session info with current date
            session_info = {"last_auth_date": datetime.datetime.now().isoformat()}
            save_session_info(session_info)
            
            print("Session initialized successfully.")
            print("You can now close this browser.")
            
            # Wait a bit before closing
            time.sleep(3)
            return True
            
        except Exception as e:
            print(f"Could not confirm WhatsApp Web loaded: {str(e)}")
            return False
        
    except Exception as e:
        print(f"Error initializing session: {str(e)}")
        return False
    
    finally:
        # Close the browser
        driver.quit()

def check_and_reinitialize_session():
    """Check if session needs reinitialization and do it if necessary"""
    if not is_session_valid():
        print("WhatsApp session has expired or doesn't exist.")
        return initialize_whatsapp_session()
    return True

def send_whatsapp_message(phone_number, message):
    """
    Send WhatsApp message using a headless browser with saved session
    
    Args:
        phone_number: Phone number with country code but without + or spaces
        message: Message to send
    
    Returns:
        bool: True if message sent successfully, False otherwise
    """
    # Check and reinitialize session if needed
    if not check_and_reinitialize_session():
        print("Failed to initialize or verify session. Message not sent.")
        return False
    
    # Setup Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")  # New headless mode
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("user-data-dir=./chrome_profile")
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    
    # Initialize the driver
    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        # URL encode the message
        encoded_message = urllib.parse.quote(message)
        
        # Create the WhatsApp URL with the phone number and message
        url = f"https://web.whatsapp.com/send?phone={phone_number}&text={encoded_message}"
        
        # Open the URL
        driver.get(url)
        
        print(f"Sending message to {phone_number}...")
        
        # Wait for the message input box to load
        input_box = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "//div[@contenteditable='true'][@data-tab='10']"))
        )
        
        # Check if message is already present
        current_message = input_box.text.replace('\n', ' ').strip()
        if current_message != message.strip():
            # Clear and resend message if not present
            input_box.clear()
            input_box.send_keys(message)
        
        # Wait for send button to be clickable
        send_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Send']"))
        )
        
        # Click the send button
        send_button.click()
        
        # Wait a moment for the message to be sent
        time.sleep(2)
        
        print(f"Message sent to {phone_number} successfully!")
        return True
        
    except Exception as e:
        print(f"Error sending message: {str(e)}")
        # Take screenshot for debugging
        driver.save_screenshot('error_screenshot.png')
        # If we get authentication error, let's invalidate the session
        if "auth" in str(e).lower() or "login" in str(e).lower():
            print("Authentication issue detected. Session might be expired.")
            # Force session reinitialization next time
            session_info = {"last_auth_date": None}
            save_session_info(session_info)
        return False
    
    finally:
        # Close the browser
        driver.quit()

# Example of usage remains the same

In [15]:
import pywhatkit as pwk

# Send message instantly
ph_no = "+917875097430"  # Include country code with +
message = "Test!!!"

try:
    # Send message immediately (will open browser if not already logged in)
    pwk.sendwhatmsg_instantly(
        phone_no=ph_no,
        message=message,
        tab_close=True,  # Close browser tab after sending
    )
    print("Message sent successfully!")
except Exception as e:
    print(f"Error sending message: {str(e)}")

Message sent successfully!


In [20]:
# Dependencies
import time
import urllib.parse
import os
import json
import datetime
import pywhatkit as pwk

# Constants
SESSION_INFO_FILE = "whatsapp_session_info.json"
SESSION_EXPIRY_DAYS = 14  # WhatsApp sessions typically last around 14 days

def get_session_info():
    """Get WhatsApp session information from the JSON file"""
    if os.path.exists(SESSION_INFO_FILE):
        try:
            with open(SESSION_INFO_FILE, 'r') as file:
                return json.load(file)
        except:
            return {"last_auth_date": None}
    return {"last_auth_date": None}

def save_session_info(session_info):
    """Save WhatsApp session information to the JSON file"""
    with open(SESSION_INFO_FILE, 'w') as file:
        json.dump(session_info, file)

def is_session_valid():
    """Check if the WhatsApp session is still valid"""
    session_info = get_session_info()
    last_auth_date = session_info.get("last_auth_date")
    
    if not last_auth_date:
        return False
    
    # Convert string date to datetime object
    last_auth = datetime.datetime.fromisoformat(last_auth_date)
    
    # Check if session is expired (older than SESSION_EXPIRY_DAYS)
    current_date = datetime.datetime.now()
    delta = current_date - last_auth
    
    return delta.days < SESSION_EXPIRY_DAYS

def update_session_info():
    """Update the session information after successful login"""
    session_info = {"last_auth_date": datetime.datetime.now().isoformat()}
    save_session_info(session_info)
    print("Session information updated.")

def check_and_reinitialize_session():
    """Check if session needs reinitialization"""
    if not is_session_valid():
        print("WhatsApp session has expired or doesn't exist.")
        return False
    return True

def send_whatsapp_message(phone_number, message):
    """
    Send WhatsApp message using pywhatkit
    
    Args:
        phone_number: Phone number with country code in format "+XXXXXXXXXXXX"
        message: Message to send
        
    Returns:
        bool: True if message sent successfully, False otherwise
    """
    try:
        # Check if our session tracking thinks the session is valid
        if not check_and_reinitialize_session():
            print("Session may be expired. PyWhatKit will open a new browser window.")
            # We'll update the session info after successful sending
        
        # Send message immediately using pywhatkit
        print(f"Sending message to {phone_number}...")
        pwk.sendwhatmsg_instantly(
            phone_no=phone_number,
            message=message,
            tab_close=True,  # Close browser tab after sending
        )
        
        # Update session info after successful sending
        update_session_info()
        print(f"Message sent to {phone_number} successfully!")
        return True
        
    except Exception as e:
        print(f"Error sending message: {str(e)}")
        return False

def send_scheduled_message(phone_number, message, hours=None, minutes=None):
    """
    Schedule a WhatsApp message to be sent at a specific time
    
    Args:
        phone_number: Phone number with country code in format "+XXXXXXXXXXXX"
        message: Message to send
        hours: Hour to send (24-hour format, default is current hour)
        minutes: Minutes to send (default is current minute + 1)
    """
    try:
        # Set default time to current + 1 minute if not specified
        if hours is None or minutes is None:
            now = datetime.datetime.now()
            hours = hours or now.hour
            minutes = minutes or now.minute + 1
            
            # Adjust hours if minutes exceed 59
            if minutes >= 60:
                hours = (hours + 1) % 24
                minutes = minutes % 60
        
        # Schedule the message
        print(f"Scheduling message to {phone_number} at {hours}:{minutes}...")
        pwk.sendwhatmsg(
            phone_no=phone_number,
            message=message,
            time_hour=hours,
            time_min=minutes,            
            tab_close=True
        )
        
        # Update session information
        update_session_info()
        print(f"Message to {phone_number} scheduled successfully!")
        return True
        
    except Exception as e:
        print(f"Error scheduling message: {str(e)}")
        return False

# Example usage
def main():
    phone_number = "+917875097430"  # Include country code with +
    message = "Test message from WhatsApp automation script!"
    
    # Option 1: Send message immediately
    send_whatsapp_message(phone_number, message)
    
    # Option 2: Schedule message for specific time
    # now = datetime.datetime.now()
    # send_scheduled_message(phone_number, message, now.hour, now.minute + 2)
    
    # Option 3: Send message to a group (if needed)
    # group_id = "FamilyGroup"  # The group ID from the WhatsApp group invite link
    # pwk.sendwhatmsg_to_group(group_id, "Hello Group!", now.hour, now.minute + 3)

if __name__ == "__main__":
    main()

Sending message to +917875097430...
Session information updated.
Message sent to +917875097430 successfully!


This Jupyter Notebook demonstrates various methods to send WhatsApp messages using different Python libraries and techniques. Below are the key components and functionalities included in the notebook:

1. **Installing Required Packages**:
	- `pywhatkit`
	- `selenium`

2. **Importing Dependencies**:
	- `pywhatkit` for sending WhatsApp messages.
	- `selenium` for browser automation.
	- `pyautogui` for desktop automation.
	- Other standard libraries like `time`, `datetime`, `sys`, `webbrowser`, `subprocess`, and `os`.

3. **Defining Constants and Variables**:
	- `contacts`: List of contact names.
	- `message`: Message to be sent.
	- `phone_no_list`: List of phone numbers.
	- `msg`: Another message variable.
	- `now`: Current datetime.
	- `_hr` and `_min`: Current hour and minute adjusted for scheduling messages.

4. **Functions for Sending Messages**:
	- `click_search_name(name)`: Uses `pyautogui` to find and click the search bar, then type a name.
	- `click_send_message(msg)`: Uses `pyautogui` to find and click the message box, then type and send a message.
	- `send_scheduled_message()`: Uses `pywhatkit` to send a WhatsApp message at a scheduled time.
	- `send_instant_messages()`: Uses `pywhatkit` to send instant messages to a list of phone numbers.
	- `send_whatsapp_message(phone_number, message)`: Uses `selenium` to send a WhatsApp message without visibly opening a browser.
	- `send_message_via_whatsapp_desktop(contact_name, message)`: Uses `pyautogui` to send a WhatsApp message through the desktop application.

5. **Main Execution**:
	- Options for sending WhatsApp messages using the defined functions.
	- Example usage of the functions to send messages.

The notebook provides a comprehensive guide to automate WhatsApp messaging using Python, covering both web-based and desktop-based approaches.

In [ ]:
# Dependencies
import pyautogui
import time
import subprocess
import os

def send_message_via_whatsapp_desktop(contact_name, message):
    """
    Send WhatsApp message through the desktop application
    
    Args:
        contact_name: Name of the contact exactly as it appears in WhatsApp
        message: Message to send
    """
    try:
        # Path to WhatsApp desktop app - update this for your system
        whatsapp_path = "C:\\Users\\YourUsername\\AppData\\Local\\WhatsApp\\WhatsApp.exe"
        
        # Start WhatsApp
        subprocess.Popen(whatsapp_path)
        
        # Wait for WhatsApp to open
        time.sleep(5)
        
        # Click on the search bar
        search_position = pyautogui.locateOnScreen('search_icon.png', confidence=0.7)
        if search_position:
            search_x, search_y = pyautogui.center(search_position)
            pyautogui.click(search_x, search_y)
        else:
            # Fallback to likely position of search bar
            pyautogui.click(200, 115)
        
        time.sleep(1)
        
        # Type the contact name
        pyautogui.write(contact_name)
        time.sleep(2)
        
        # Press Enter to select the contact
        pyautogui.press('enter')
        time.sleep(1)
        
        # Type message
        pyautogui.write(message)
        time.sleep(1)
        
        # Send message
        pyautogui.press('enter')
        
        print(f"Message sent to {contact_name}")
        
        # Optional: Close WhatsApp after sending
        # time.sleep(5)
        # pyautogui.hotkey('alt', 'f4')
        
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    contact_name = "Varun"
    message = "Test message from Python automation"
    
    send_message_via_whatsapp_desktop(contact_name, message)